In [1]:
# Prepare the MPST corpus to convert the plot synopses into sequences

In [46]:
EXAMPLE_ID = 'tt0059800'

In [47]:
import os
import json
import torch
import operator
import numpy as np

In [48]:
DATA_PATH = '../../data/MPST/'
PROCESSED_DATA_PATH = '../../processed_data/new/'

In [49]:
os.listdir(DATA_PATH)

['LICENSE',
 'final_plots_wiki_imdb_combined',
 'tag_assignment_data',
 'README.md',
 'train_ids.txt',
 'test_ids.txt',
 'id_title_tags.tsv']

In [50]:
train_ids = open(DATA_PATH + 'train_ids.txt').read().split('\n')[:-1]
test_ids = open(DATA_PATH + 'test_ids.txt').read().split('\n')[:-1]

print('Train %d + Test %d = Total %d' % (len(train_ids), len(test_ids), (len(train_ids) + len(test_ids))))

Train 11862 + Test 2966 = Total 14828


In [51]:
print(train_ids[:5])

['tt0059800', 'tt0042296', 'tt0965375', 'tt2199711', 'tt0057091']


In [52]:
all_imdb_list = train_ids + test_ids
len(all_imdb_list)

14828

In [56]:
#import random
#random.shuffle(train_ids)
len_of_train = round(len(train_ids)*0.8)
partition = {'train':train_ids[:len_of_train], 'test':test_ids, 'validation':train_ids[len_of_train:]}
json.dump(partition,open(PROCESSED_DATA_PATH + 'partition.json','w'))

# Process Tags

In [9]:
tag_list = open( DATA_PATH + 'tag_assignment_data/tag_list.txt').read().split('\n')[:-1]
print(len(tag_list))

71


In [10]:
index_to_tag = {i: tag for i, tag in enumerate(tag_list)}
tag_to_index = {tag: i for i, tag in enumerate(tag_list)}

In [11]:
json.dump(index_to_tag, open(PROCESSED_DATA_PATH + 'index_to_tag.json', 'w'))
json.dump(tag_to_index, open(PROCESSED_DATA_PATH + 'tag_to_index.json', 'w'))

In [12]:
movie_to_tags = json.load( open(DATA_PATH + 'tag_assignment_data/movie_to_label_name.json') )
print(movie_to_tags[train_ids[0]])

['cult', 'humor', 'murder', 'suspenseful']


In [13]:
imdb_to_binary_tag_dict = {}

# Train
#全ての映画の正解ラベルの生成
for imdb_id in all_imdb_list:
    tags = movie_to_tags[imdb_id]
    binary_vector = [0] * len(tag_list)
    for t in tags:
        binary_vector[tag_to_index[t]] = 1
    imdb_to_binary_tag_dict[imdb_id] = binary_vector


In [14]:
json.dump(imdb_to_binary_tag_dict, open(PROCESSED_DATA_PATH + 'vectors/labels_binary_dict.json', 'w'))

# Process Plot Synopses
## Add stopword removal or other preprocessing later

In [15]:
all_texts_cleaned = {}

In [16]:
words = {}

for imdb_id in all_imdb_list:
    plot_words = open(DATA_PATH + '/final_plots_wiki_imdb_combined/cleaned/{}.txt'.format(imdb_id)).read().lower().split()
    all_texts_cleaned[imdb_id] = plot_words
    for w in plot_words:
        if w not in words:
            words[w] = 0
        words[w] += 1

In [17]:
words_sorted_by_frequency = sorted(words.items(), key = operator.itemgetter(1), reverse=True)

In [18]:
len(words_sorted_by_frequency)

112358

In [21]:
#print(len([v for k, v in words_sorted_by_frequency if v>10]))
print([(k, v) for k, v in words_sorted_by_frequency if v<=100])

[('inspects', 100), ('apply', 100), ('propaganda', 100), ('ludlow', 100), ('enlisted', 100), ('bends', 100), ('ignites', 100), ('summit', 100), ('hi', 100), ('software', 100), ('values', 100), ('weekly', 100), ('proprietor', 100), ('orcs', 100), ('mutters', 100), ('remnants', 100), ('spoon', 100), ('enrages', 100), ('monroe', 100), ('hint', 100), ('yards', 100), ('sexuality', 100), ('chute', 100), ('hatchet', 100), ('raining', 100), ('gladys', 100), ('separation', 100), ('fooled', 100), ('arranging', 100), ('plug', 100), ('regime', 100), ('clever', 100), ('hooked', 100), ('risks', 100), ('blackmailing', 100), ('horizon', 100), ('abduct', 100), ('dishes', 100), ('1969', 100), ('celebrities', 100), ('greenhouse', 100), ('splinter', 100), ('drilling', 100), ('whitney', 100), ('directing', 100), ('miniature', 100), ('winnings', 100), ('difficulties', 100), ('geek', 100), ('cleaver', 100), ('sobs', 100), ('susanna', 100), ('ethel', 100), ('dumb', 100), ('missy', 100), ('trey', 100), ('ka', 

In [22]:
VOCAB_5K_FREQ = words_sorted_by_frequency[:5000]
VOCAB_5K = [word for word, freq in VOCAB_5K_FREQ]
word2idx = {w: i+1 for i, w in enumerate(VOCAB_5K)}
idx2word = {i+1: w for i, w in enumerate(VOCAB_5K)}
json.dump(VOCAB_5K, open(PROCESSED_DATA_PATH + 'vocab_5k_no_process.json', 'w'))
json.dump(word2idx, open(PROCESSED_DATA_PATH + 'word2idx_no_process.json', 'w'))
json.dump(idx2word, open(PROCESSED_DATA_PATH + 'idx2word_no_process.json', 'w'))

In [19]:
VOCAB_NEW_FREQ = words_sorted_by_frequency[:31517]
VOCAB_31k = [w for w, f in VOCAB_NEW_FREQ]
word2idx = {w: i+1 for i, w in enumerate(VOCAB_31k)}
idx2word = {i+1: w for i, w in enumerate(VOCAB_31k)}
json.dump(VOCAB_31k, open(PROCESSED_DATA_PATH + 'vocab_31k_no_process.json', 'w'))
json.dump(word2idx, open(PROCESSED_DATA_PATH + 'word2idx_no_process.json', 'w'))
json.dump(idx2word, open(PROCESSED_DATA_PATH + 'idx2word_no_process.json', 'w'))

## Convert Text to Sequence

In [24]:
# Load clean texts into dictionary
clean_text_dict = {}
for imdb_id in all_imdb_list:
    clean_text_dict[imdb_id] = open(DATA_PATH + '/final_plots_wiki_imdb_combined/cleaned/{}.txt'.format(imdb_id)).read().lower().split()

In [27]:
print(clean_text_dict[EXAMPLE_ID])

['james', 'bond', 'attends', 'the', 'funeral', 'of', 'colonel', 'jacques', 'bouvar', 'a', 'spectre', 'operative', 'bouvar', 'is', 'alive', 'and', 'disguised', 'as', 'his', 'own', 'widow', 'but', 'bond', 'identifies', 'him', 'following', 'him', 'to', 'a', 'château', 'bond', 'fights', 'and', 'kills', 'him', 'escaping', 'using', 'a', 'jetpack', 'and', 'his', 'aston', 'martin', 'db5', 'conveniently', 'driven', 'up', 'by', 'french', 'agent', 'madame', 'la', 'porte', 'at', 'a', 'spectre', 'meeting', 'in', 'paris', 'chaired', 'by', 'the', 'enigmatic', 'no', '1', 'emilio', 'largo', 'introduces', 'the', 'group', 's', 'latest', 'project', 'stealing', 'two', 'atomic', 'bombs', 'and', 'holding', 'nato', 'for', 'ransom', 'the', 'plot', 'begins', 'at', 'the', 'shrublands', 'sanatorium', 'located', 'close', 'to', 'an', 'air', 'force', 'base', 'coincidentally', 'james', 'bond', 'is', 'at', 'shrublands', 'to', 'improve', 'his', 'health', 'there', 'he', 'notices', 'count', 'lippe', 'a', 'suspicious', 'm

In [28]:
def text_to_sequence(word2idx, text):
    sequence = [word2idx[w] for w in plot_words if w in word2idx]
    return sequence

In [29]:
def get_sequence_len_stat(sequences):
    lens = [len(s) for s in sequences]
    print('Total %d, Min %d, Max %d, Mean %d, Median %d' %(len(lens), min(lens), max(lens), np.mean(lens), np.median(lens)))

In [30]:
all_sequences = {}
long_count = 0
for imdb_id, plot_words in all_texts_cleaned.items():
    sequence = text_to_sequence(word2idx, plot_words)
    all_sequences[imdb_id] = sequence
    if len(sequence) > 3000:
        long_count += 1
    
get_sequence_len_stat(list(all_sequences.values()))

json.dump(all_sequences, open( PROCESSED_DATA_PATH + 'all_sequence_dict.json', 'w') )
print('Dumped')

Total 14828, Min 19, Max 10426, Mean 780, Median 566
Dumped


In [31]:
all_sequences[EXAMPLE_ID]

[564,
 587,
 2620,
 1,
 1054,
 5,
 1129,
 4,
 6,
 454,
 3,
 2362,
 17,
 9,
 134,
 2474,
 21,
 587,
 2978,
 15,
 368,
 15,
 2,
 4,
 587,
 1041,
 3,
 203,
 15,
 1972,
 295,
 4,
 3,
 9,
 786,
 1847,
 32,
 20,
 933,
 507,
 2861,
 1079,
 22,
 4,
 418,
 8,
 966,
 20,
 1,
 93,
 1021,
 1025,
 1,
 159,
 11,
 2253,
 1435,
 1675,
 53,
 3223,
 3,
 683,
 16,
 2672,
 1,
 1034,
 120,
 22,
 1,
 2570,
 488,
 2,
 29,
 571,
 478,
 932,
 564,
 587,
 6,
 22,
 2,
 9,
 2206,
 64,
 7,
 458,
 1407,
 4,
 1153,
 54,
 13,
 4,
 4095,
 587,
 2058,
 11,
 86,
 21,
 6,
 323,
 210,
 20,
 11,
 3042,
 623,
 137,
 6,
 1414,
 8,
 100,
 2,
 316,
 587,
 13,
 4,
 640,
 21,
 587,
 6,
 1163,
 20,
 9,
 326,
 7,
 38,
 33,
 136,
 363,
 3,
 1,
 54,
 28,
 324,
 5,
 11,
 1034,
 11,
 216,
 37,
 2,
 1281,
 1,
 1726,
 5,
 4,
 933,
 571,
 478,
 1210,
 2,
 1013,
 18,
 4,
 1225,
 614,
 1551,
 29,
 2969,
 3575,
 13,
 53,
 3223,
 6,
 684,
 20,
 25,
 38,
 100,
 2,
 104,
 139,
 16,
 9,
 3255,
 24,
 507,
 3401,
 1316,
 2,
 9,
 554,
 3,
 7,
 84,

In [24]:
long_count

451

# Pad sequences

The lists of sequences dumped have the same order of the ids we had in the train and test id files

In [25]:
def pad_one_sequence(seq,  max_len=1500, direction='left', value=0):
    seq_len = len(seq)
    pad_len = abs(max_len - seq_len)
    padded_sequence = [0] * max_len
    
    #if seq_len < max_len:
    if seq_len <= max_len:
        padding = [value] * (pad_len)
        if direction == 'left':
            padded_seq = padding + seq
        elif direction == 'right':
            padded_seq = seq + padding
        else:
            raise ValueError("Check the value of arguement 'direction'")
                
    elif seq_len > max_len:
        # Truncate
        if direction == 'left':
            padded_seq = seq[-max_len : ]
        elif direction == 'right':
            padded_seq = seq[: max_len]
        else:
            raise ValueError("Check the value of arguement 'direction'")
    
    return padded_seq

In [26]:
padded_seq_dict = {}
max_len = 1500
direction = 'left'
value = 0

for imdb_id, sequence in all_sequences.items():
    padded_seq_dict[imdb_id] = pad_one_sequence(sequence, max_len, direction, value)

print(len(padded_seq_dict), len(padded_seq_dict[imdb_id]))
json.dump(padded_seq_dict, open(PROCESSED_DATA_PATH + 'vectors/padded_word_sequences_1500.json', 'w'))

14828 1500


# Compute and Dump Emotion Flow Data

In [34]:
import joblib

# NRC emotion lexicons as dictionary format. Key: Emotion type, Value: List of words
emotion_lexicons = joblib.load(PROCESSED_DATA_PATH + 'emotion_lexicons_dict.pkl')
emotion_types = ['anger', 'anticipation', 'disgust', 'fear', 'joy',
                     'negative', 'positive', 'sadness', 'surprise', 'trust']

# Get shortest and longest words length in the lexicon list
minl= float('inf')
maxl = -1
imdb_ids = []
segment_scores_list = []
plot_lengths = []
print(list(emotion_lexicons))
for k, v in emotion_lexicons.items():
    for word in v:
        minl = min(minl,len(word))
        maxl = max(maxl,len(word))

print(minl, maxl)

['positive', 'sadness', 'joy', 'trust', 'fear', 'negative', 'surprise', 'anger', 'anticipation', 'disgust']
2 17


In [28]:
def get_chunks_of_emotional_scores(content, N=20):
    # TODO: STEMMING BASED
    # init scores
    segment_scores =[[0 for j in range(len(emotion_types))] for i in range(N)]

    # content = content.lower().split()
    segment_length = round(len(content) / N)

    for i in range(N):
        start_index = i * segment_length
        end_index   = (i+1) * segment_length

        for token in content[start_index:end_index]:
            if len(token)>= minl and len(token) <= maxl:
                for emo_idx in range(len(emotion_types)):
                    if token in  emotion_lexicons[emotion_types[emo_idx]]:
                        segment_scores[i][emo_idx] += 1

        total = sum(segment_scores[i])
        if total > 0:
            for j in range(len(segment_scores[i])):
                segment_scores[i][j] /= total
                segment_scores[i][j] *= 100

    return segment_scores

In [29]:
emotion_score_vectors_dict = {}

for imdb_id, clean_content in clean_text_dict.items():
    emotion_score_vectors_dict[imdb_id] = get_chunks_of_emotional_scores(clean_content, 20)

print(len(emotion_score_vectors_dict), len(emotion_score_vectors_dict[imdb_id]) )

14828 20


In [30]:
json.dump(emotion_score_vectors_dict, open(PROCESSED_DATA_PATH + 'vectors/emotion_score_dict_20_chunks.json', 'w'))

# Compute and Dump Class Weights

In [31]:
def list_tag_weights():
    tag_to_n_movies = {}
    
    for imdb_id in train_ids:
        tags = movie_to_tags[imdb_id]
        for tag in tags:
            tag_to_n_movies[tag] = tag_to_n_movies.get(tag, 0) + 1
            
    weights = [0] * len(tag_list)
    
    for tag, count in tag_to_n_movies.items():
        # weights[tag_to_index[tag]] = (len(train_ids) - count) / len(train_ids)
        # weights[tag_to_index[tag]] = len(train_ids)/ (count * len(train_ids))
        weights[tag_to_index[tag]] = len(train_ids)/ (71 * count)
    
    print(weights)
    json.dump(weights, open(PROCESSED_DATA_PATH + 'class_weights_sk.json', 'w'))

In [32]:
list_tag_weights()

[0.03610772045282284, 0.04753070342395769, 0.07136711769979123, 0.07204416668185049, 0.07970917105687561, 0.0851965438731317, 0.11123197239361603, 0.11205259727378354, 0.19472077218556091, 0.24389842705870257, 0.2570314192849404, 0.2598295840360984, 0.28317020768679874, 0.2784507042253521, 0.3122811636172173, 0.33148893360160964, 0.3341408450704225, 0.34951971241676, 0.37044439586521344, 0.4006484952882764, 0.4074888354517348, 0.4155980660079882, 0.4240366054193179, 0.45898467729453646, 0.4842620943049602, 0.46667715791958453, 0.5078128344535296, 0.5220950704225352, 0.527036033234105, 0.560639001796011, 0.6875326030255607, 0.7458501006036218, 0.8032231852654388, 0.7807029090430433, 0.8189726594863297, 0.9179693545890729, 0.8701584507042254, 0.9546881287726359, 0.9770200148257969, 0.9827671913835957, 1.0249719173939342, 1.0377044877963433, 1.0004216918276123, 1.0709642470205851, 1.106426639306035, 1.305237676056338, 1.2375586854460094, 1.3473421172194457, 1.33656338028169, 1.41585103843

In [33]:
list_tag_weights()

[0.03610772045282284, 0.04753070342395769, 0.07136711769979123, 0.07204416668185049, 0.07970917105687561, 0.0851965438731317, 0.11123197239361603, 0.11205259727378354, 0.19472077218556091, 0.24389842705870257, 0.2570314192849404, 0.2598295840360984, 0.28317020768679874, 0.2784507042253521, 0.3122811636172173, 0.33148893360160964, 0.3341408450704225, 0.34951971241676, 0.37044439586521344, 0.4006484952882764, 0.4074888354517348, 0.4155980660079882, 0.4240366054193179, 0.45898467729453646, 0.4842620943049602, 0.46667715791958453, 0.5078128344535296, 0.5220950704225352, 0.527036033234105, 0.560639001796011, 0.6875326030255607, 0.7458501006036218, 0.8032231852654388, 0.7807029090430433, 0.8189726594863297, 0.9179693545890729, 0.8701584507042254, 0.9546881287726359, 0.9770200148257969, 0.9827671913835957, 1.0249719173939342, 1.0377044877963433, 1.0004216918276123, 1.0709642470205851, 1.106426639306035, 1.305237676056338, 1.2375586854460094, 1.3473421172194457, 1.33656338028169, 1.41585103843

# Get Pretrained Embeddings

In [35]:
import gensim
def get_pretrained_embeddings(vocabulary_path, pre_trained_vector_path, dump_path, vector_dim=300, type='fasttext'):
    # Load vocabulary
    words = json.load(open(vocabulary_path, 'r'))
    vocab_size = len(words) + 1
    print('vocabulary size', vocab_size)

    # Load word embeddings and create embedding matrix
    print('Loading Embeddings')

    if type == 'fasttext':
        model= gensim.models.wrappers.FastText.load_fasttext_format(pre_trained_vector_path)
    elif type == 'word2vec':
        model = gensim.models.KeyedVectors.load_word2vec_format(pre_trained_vector_path, binary=True)

    print('Loaded')
    print(model['try'])

    embedding_matrix = np.zeros((vocab_size, vector_dim))
    counter = 0

    for i, word in words.items():
        embedding_vector = np.zeros(vector_dim)
        if word in model:
            print(i, counter)
            embedding_matrix[int(i)] = model[word]
            counter += 1
        else:
            embedding_matrix[int(i)] = embedding_vector

    json.dump(embedding_matrix.tolist(), open(dump_path, 'w'))

    print(len(embedding_matrix))
    print(embedding_matrix.shape)
    print('Coverage', (counter/vocab_size))

In [45]:
emb_vocab_path = PROCESSED_DATA_PATH + '/idx2word_no_process.json'
#emb_vocab_path = '/home/sk/SK/Works/NarrativeAnalysis/experiments/classification/features/sequence/vocabulary_index_to_word.json'
# Replace this path
#pre_trained_vector_path = '/home/sk/SK/Works/resources/fasttext/wiki.en/wiki.en'
pre_trained_vector_path = '../../extra/fasttext/wiki.en'
dump_path = PROCESSED_DATA_PATH + 'fasttext_vectors_5k_paper_model.json'
#dump_path = '/home/sk/SK/Works/NarrativeAnalysis/experiments/classification/features/sequence/fasttext_vectors_5k_paper_model.json'

get_pretrained_embeddings(emb_vocab_path, pre_trained_vector_path, dump_path)

vocabulary size 31518
Loading Embeddings
Loaded
[-0.06831407  0.30915412  0.12180331 -0.22247837 -0.47748762 -0.12033118
 -0.19960941 -0.10533361 -0.2503069   0.19360551  0.18619625  0.1456981
 -0.281463   -0.00504994  0.4646648   0.22578108  0.19691055  0.23636696
  0.03521357  0.06550668 -0.18728039  0.12686458  0.0613632  -0.21359481
 -0.52767116 -0.30925015 -0.22214665 -0.14821781 -0.44144124  0.29986697
 -0.01606236  0.11623905 -0.28910562  0.2665176  -0.04483953 -0.2929797
  0.11571283  0.09822313  0.1814719  -0.22369501  0.00893341 -0.16172571
 -0.03157985  0.15298676  0.01418889  0.43996707 -0.01828363  0.02063813
 -0.36280861 -0.254769    0.02717102 -0.15333425  0.03007952 -0.1372713
 -0.24108402  0.29698482 -0.00568777 -0.11870692 -0.0259715   0.1788373
 -0.03861186 -0.22213067  0.30692723  0.16983613 -0.08305422 -0.01824462
  0.00250198  0.38897112 -0.22285722  0.05966283  0.03053282  0.08914272
 -0.09346877 -0.34166712  0.05875046  0.07086216  0.33161137  0.38900018
  0.102

784 782
785 783
786 784
787 785
788 786
789 787
790 788
791 789
792 790
793 791
794 792
795 793
796 794
797 795
798 796
799 797
800 798
801 799
802 800
803 801
804 802
805 803
806 804
807 805
808 806
809 807
810 808
811 809
812 810
813 811
814 812
815 813
816 814
817 815
818 816
819 817
820 818
821 819
822 820
823 821
824 822
825 823
826 824
827 825
828 826
829 827
830 828
831 829
832 830
833 831
834 832
835 833
836 834
837 835
838 836
839 837
840 838
841 839
842 840
843 841
844 842
845 843
846 844
847 845
848 846
849 847
850 848
851 849
852 850
853 851
854 852
855 853
856 854
857 855
858 856
859 857
860 858
861 859
862 860
863 861
864 862
865 863
866 864
867 865
868 866
869 867
870 868
871 869
872 870
873 871
874 872
875 873
876 874
877 875
878 876
879 877
880 878
881 879
882 880
883 881
884 882
885 883
886 884
887 885
888 886
889 887
890 888
891 889
892 890
893 891
894 892
895 893
896 894
897 895
898 896
899 897
900 898
901 899
902 900
903 901
904 902
905 903
906 904
907 905
908 906


1810 1802
1811 1803
1812 1804
1813 1805
1814 1806
1815 1807
1816 1808
1817 1809
1818 1810
1819 1811
1820 1812
1821 1813
1822 1814
1823 1815
1824 1816
1825 1817
1826 1818
1827 1819
1828 1820
1829 1821
1830 1822
1831 1823
1832 1824
1833 1825
1834 1826
1835 1827
1836 1828
1837 1829
1838 1830
1839 1831
1840 1832
1841 1833
1842 1834
1843 1835
1844 1836
1845 1837
1846 1838
1847 1839
1848 1840
1849 1841
1850 1842
1851 1843
1852 1844
1853 1845
1854 1846
1855 1847
1856 1848
1857 1849
1858 1850
1859 1851
1860 1852
1861 1853
1862 1854
1863 1855
1864 1856
1865 1857
1866 1858
1867 1859
1868 1860
1869 1861
1870 1862
1871 1863
1872 1864
1873 1865
1874 1866
1875 1867
1876 1868
1877 1869
1878 1870
1879 1871
1880 1872
1881 1873
1882 1874
1883 1875
1884 1876
1885 1877
1887 1878
1888 1879
1889 1880
1890 1881
1891 1882
1892 1883
1893 1884
1894 1885
1895 1886
1896 1887
1897 1888
1898 1889
1899 1890
1900 1891
1901 1892
1902 1893
1903 1894
1904 1895
1905 1896
1906 1897
1907 1898
1908 1899
1909 1900
1910 1901


2828 2809
2829 2810
2830 2811
2831 2812
2832 2813
2833 2814
2834 2815
2835 2816
2836 2817
2837 2818
2838 2819
2839 2820
2840 2821
2841 2822
2842 2823
2843 2824
2844 2825
2845 2826
2846 2827
2847 2828
2848 2829
2849 2830
2850 2831
2851 2832
2852 2833
2853 2834
2854 2835
2855 2836
2856 2837
2857 2838
2858 2839
2859 2840
2860 2841
2861 2842
2862 2843
2863 2844
2864 2845
2865 2846
2866 2847
2867 2848
2868 2849
2869 2850
2870 2851
2871 2852
2872 2853
2873 2854
2874 2855
2875 2856
2876 2857
2877 2858
2878 2859
2879 2860
2880 2861
2881 2862
2882 2863
2883 2864
2884 2865
2885 2866
2886 2867
2887 2868
2888 2869
2889 2870
2890 2871
2891 2872
2892 2873
2893 2874
2894 2875
2895 2876
2896 2877
2897 2878
2898 2879
2899 2880
2900 2881
2901 2882
2902 2883
2903 2884
2904 2885
2905 2886
2906 2887
2907 2888
2908 2889
2909 2890
2910 2891
2911 2892
2912 2893
2913 2894
2914 2895
2915 2896
2916 2897
2917 2898
2918 2899
2919 2900
2920 2901
2921 2902
2922 2903
2923 2904
2924 2905
2925 2906
2926 2907
2927 2908


3991 3965
3992 3966
3993 3967
3994 3968
3995 3969
3996 3970
3997 3971
3998 3972
3999 3973
4000 3974
4001 3975
4002 3976
4003 3977
4004 3978
4006 3979
4007 3980
4008 3981
4009 3982
4010 3983
4011 3984
4012 3985
4013 3986
4014 3987
4015 3988
4016 3989
4017 3990
4018 3991
4019 3992
4020 3993
4021 3994
4022 3995
4023 3996
4024 3997
4025 3998
4026 3999
4027 4000
4028 4001
4029 4002
4030 4003
4031 4004
4032 4005
4033 4006
4034 4007
4035 4008
4036 4009
4037 4010
4038 4011
4039 4012
4040 4013
4041 4014
4042 4015
4043 4016
4044 4017
4045 4018
4046 4019
4047 4020
4048 4021
4049 4022
4050 4023
4051 4024
4052 4025
4053 4026
4054 4027
4055 4028
4056 4029
4057 4030
4058 4031
4059 4032
4060 4033
4061 4034
4062 4035
4063 4036
4064 4037
4065 4038
4066 4039
4067 4040
4068 4041
4069 4042
4070 4043
4071 4044
4072 4045
4073 4046
4074 4047
4075 4048
4076 4049
4077 4050
4078 4051
4079 4052
4080 4053
4081 4054
4082 4055
4083 4056
4084 4057
4085 4058
4086 4059
4087 4060
4088 4061
4090 4062
4091 4063
4092 4064


...

27524 27276
27525 27277
27526 27278
27527 27279
27528 27280
27529 27281
27530 27282
27531 27283
27532 27284
27533 27285
27534 27286
27535 27287
27536 27288
27537 27289
27538 27290
27539 27291
27540 27292
27541 27293
27542 27294
27543 27295
27544 27296
27545 27297
27546 27298
27547 27299
27548 27300
27549 27301
27550 27302
27551 27303
27552 27304
27553 27305
27554 27306
27555 27307
27556 27308
27557 27309
27558 27310
27559 27311
27560 27312
27561 27313
27562 27314
27563 27315
27564 27316
27565 27317
27566 27318
27567 27319
27568 27320
27569 27321
27570 27322
27571 27323
27572 27324
27573 27325
27574 27326
27575 27327
27576 27328
27577 27329
27578 27330
27579 27331
27580 27332
27581 27333
27582 27334
27583 27335
27584 27336
27585 27337
27586 27338
27587 27339
27588 27340
27589 27341
27590 27342
27591 27343
27592 27344
27593 27345
27594 27346
27595 27347
27596 27348
27597 27349
27598 27350
27599 27351
27600 27352
27601 27353
27602 27354
27603 27355
27604 27356
27605 27357
27606 27358
2760

28468 28210
28469 28211
28470 28212
28471 28213
28472 28214
28473 28215
28474 28216
28475 28217
28476 28218
28477 28219
28478 28220
28479 28221
28480 28222
28481 28223
28482 28224
28483 28225
28484 28226
28485 28227
28486 28228
28487 28229
28488 28230
28489 28231
28490 28232
28491 28233
28492 28234
28493 28235
28494 28236
28495 28237
28496 28238
28497 28239
28498 28240
28499 28241
28500 28242
28501 28243
28502 28244
28503 28245
28504 28246
28505 28247
28506 28248
28507 28249
28508 28250
28509 28251
28510 28252
28511 28253
28512 28254
28513 28255
28515 28256
28516 28257
28517 28258
28518 28259
28519 28260
28520 28261
28521 28262
28522 28263
28523 28264
28524 28265
28525 28266
28526 28267
28527 28268
28528 28269
28529 28270
28530 28271
28531 28272
28532 28273
28533 28274
28534 28275
28535 28276
28536 28277
28537 28278
28538 28279
28539 28280
28540 28281
28541 28282
28542 28283
28543 28284
28544 28285
28545 28286
28546 28287
28547 28288
28548 28289
28550 28290
28551 28291
28552 28292
2855

29386 29119
29387 29120
29388 29121
29389 29122
29390 29123
29391 29124
29392 29125
29393 29126
29394 29127
29395 29128
29396 29129
29397 29130
29398 29131
29399 29132
29400 29133
29401 29134
29402 29135
29403 29136
29404 29137
29405 29138
29406 29139
29407 29140
29408 29141
29409 29142
29410 29143
29411 29144
29412 29145
29413 29146
29414 29147
29415 29148
29416 29149
29417 29150
29418 29151
29419 29152
29420 29153
29421 29154
29422 29155
29423 29156
29424 29157
29425 29158
29426 29159
29427 29160
29428 29161
29429 29162
29430 29163
29431 29164
29432 29165
29433 29166
29434 29167
29435 29168
29436 29169
29437 29170
29438 29171
29439 29172
29440 29173
29441 29174
29442 29175
29443 29176
29444 29177
29445 29178
29446 29179
29447 29180
29448 29181
29449 29182
29450 29183
29451 29184
29452 29185
29453 29186
29454 29187
29455 29188
29456 29189
29457 29190
29458 29191
29459 29192
29460 29193
29461 29194
29462 29195
29463 29196
29464 29197
29465 29198
29466 29199
29467 29200
29468 29201
2946

30103 29831
30104 29832
30105 29833
30106 29834
30107 29835
30108 29836
30109 29837
30110 29838
30111 29839
30112 29840
30113 29841
30114 29842
30115 29843
30116 29844
30117 29845
30118 29846
30119 29847
30120 29848
30121 29849
30122 29850
30124 29851
30125 29852
30126 29853
30127 29854
30128 29855
30129 29856
30130 29857
30131 29858
30132 29859
30133 29860
30134 29861
30135 29862
30136 29863
30137 29864
30138 29865
30139 29866
30140 29867
30141 29868
30142 29869
30143 29870
30144 29871
30145 29872
30146 29873
30147 29874
30148 29875
30149 29876
30150 29877
30151 29878
30152 29879
30153 29880
30154 29881
30155 29882
30156 29883
30157 29884
30158 29885
30159 29886
30160 29887
30161 29888
30162 29889
30163 29890
30164 29891
30165 29892
30166 29893
30167 29894
30168 29895
30169 29896
30170 29897
30171 29898
30172 29899
30173 29900
30174 29901
30175 29902
30176 29903
30177 29904
30178 29905
30179 29906
30180 29907
30181 29908
30182 29909
30183 29910
30184 29911
30186 29912
30187 29913
3018

30874 30593
30875 30594
30876 30595
30877 30596
30878 30597
30879 30598
30880 30599
30881 30600
30882 30601
30883 30602
30884 30603
30885 30604
30886 30605
30887 30606
30888 30607
30889 30608
30890 30609
30891 30610
30892 30611
30893 30612
30894 30613
30895 30614
30896 30615
30897 30616
30898 30617
30899 30618
30900 30619
30901 30620
30902 30621
30903 30622
30904 30623
30905 30624
30906 30625
30907 30626
30908 30627
30909 30628
30910 30629
30911 30630
30912 30631
30913 30632
30914 30633
30915 30634
30916 30635
30917 30636
30918 30637
30919 30638
30920 30639
30921 30640
30922 30641
30923 30642
30924 30643
30925 30644
30926 30645
30927 30646
30928 30647
30929 30648
30930 30649
30931 30650
30932 30651
30933 30652
30934 30653
30935 30654
30936 30655
30937 30656
30938 30657
30939 30658
30940 30659
30941 30660
30942 30661
30943 30662
30944 30663
30945 30664
30946 30665
30947 30666
30948 30667
30949 30668
30950 30669
30951 30670
30952 30671
30953 30672
30954 30673
30955 30674
30956 30675
3095

In [46]:
ft_model= gensim.models.wrappers.FastText.load_fasttext_format(pre_trained_vector_path)

In [47]:
v = ft_model['vutum']
max(v), min(v), sum(v)

(0.89575106, -1.1757601, 1.454605671693571)

In [48]:
v = ft_model['apple']
max(v), min(v), sum(v)

(0.98430365, -0.6563826, 7.806553833535872)

In [49]:
del ft_model

In [50]:
ft_model['ty']

NameError: name 'ft_model' is not defined